In [1]:
print("Hello World")

Hello World


In [4]:
import glob
import os

from liftout.model.utils import *
from liftout import model
from fibsem.detection.detection import *
from fibsem.detection.DetectionModel import *
from fibsem.detection.utils import *

import matplotlib.pyplot as plt
import numpy as np

from PIL import Image
from random import shuffle

# print(os.path.dirname(model.__file__))

model_path = os.path.dirname(model.__file__)
model_filenames = glob.glob(
    os.path.join(model_path, "models", "*.pt")
)


image_fnames = glob.glob(
    os.path.join(model_path, "data/data/train/raw/000001*.tif")
)

shuffle(image_fnames)
model_filenames = sorted([fname for fname in model_filenames if any(match in fname for match in ["boost_n05", "fresh"])]) #"n10", "baseline"
# any(x in a_string for x in matches)("boost" in fname) or ("n10" in fname)])

# there is a smarter way to do this (just order them correctly)
# model_filenames.insert(1, model_filenames[-2])
# model_filenames.insert(2, model_filenames[-1])
# model_filenames.pop()
# model_filenames.pop()
# model_filenames.pop()
print(model_filenames)


['\\\\ad.monash.edu\\home\\user007\\prcle2\\documents\\demarco\\autoliftout\\liftout\\model\\models\\boost_n05_model.pt', '\\\\ad.monash.edu\\home\\user007\\prcle2\\documents\\demarco\\autoliftout\\liftout\\model\\models\\fresh_full_n10.pt']


In [5]:

# create a grid of images with each row showing a different model

N_IMAGES = 10
EXP_NAME = input("Enter a name for the experiment")
EXP_PATH = os.path.join("eval", EXP_NAME)
os.makedirs(
    EXP_PATH, exist_ok=True)
    
image_fnames = image_fnames[:N_IMAGES]

exp_array = None
for weights_file in reversed(model_filenames):
    # load detector
    detector = Detector(weights_file=weights_file)
    detector.detection_model.model.eval()

    img_array = None

    for fname in image_fnames:
        # load image from file
        img = load_image_from_file(fname)
    
        # model inference
        mask = detector.detection_model.inference(img)

        # individual detection modes
        lamella_mask, lamella_idx = extract_class_pixels(mask, color=(255, 0, 0)) # red
        needle_mask, needle_idx = extract_class_pixels(mask, color=(0, 255, 0)) # green

        feature_1_px, lamella_centre_detection = detect_lamella_centre(img, mask) # lamella_centre
        feature_2_px, needle_tip_detection = detect_needle_tip(img, mask)
        feature_3_px, lamella_edge_detection = detect_lamella_edge(img, mask)

        mask_combined = draw_two_features(mask, feature_1_px, feature_2_px)
        img_blend = draw_overlay(img, mask_combined)
        
        if img_array is None:
            img_array = np.array(img_blend)

        else:
            img_array = np.hstack((img_array, np.array(img_blend)))

    print(f"saving: {os.path.basename(weights_file)}")
    img_pil = Image.fromarray(img_array)
    img_pil.save(os.path.join(EXP_PATH, f"{os.path.basename(weights_file)}.png"))

    if exp_array is None:
        exp_array = img_array
    else:
        exp_array = np.vstack((exp_array, img_array))

exp_img_pil = Image.fromarray(exp_array)
exp_img_pil.save(os.path.join(EXP_PATH,f"experiment.png"))
print("Finished.")

saving: fresh_full_n10.pt
saving: boost_n05_model.pt
Finished.


In [33]:

# create a grid of images from a list of image filenames
image_fnames = glob.glob(
    os.path.join(model_path, "images/test/**/*.tif")
)

shuffle(image_fnames)

imgs = [load_image_from_file(fname) for fname in image_fnames]
detector = Detector(weights_file="models/boost_n05_model.pt")
detector.detection_model.model.eval()

MAX_COLS = 10
i = 0
j = 0

full_img = None
row = None
while image_fnames:
    
    img = load_image_from_file(image_fnames.pop())
    
    # model inference
    mask = detector.detection_model.inference(img)
    feature_1_px, lamella_centre_detection = detect_lamella_centre(img, mask) # lamella_centre
    feature_2_px, needle_tip_detection = detect_needle_tip(img, mask)

    mask_combined = draw_two_features(mask, feature_1_px, feature_2_px)
    img_blend = draw_overlay(img, mask_combined)

    j += 1
    
    if row is None:
        row = img_blend
    else:
        row = np.hstack((row, img_blend))

    if j >= MAX_COLS:
        
        if full_img is None:
            full_img = row
        else:
            full_img = np.vstack((full_img, row))
        i += 1
        j = 0
        row = None

img_pil = Image.fromarray(full_img).convert("RGB")
img_pil.save("grid.png")


## Model Size


In [15]:
import glob
import os

from liftout.model.utils import *
from liftout import model
from fibsem.detection.detection import *
from fibsem.detection.DetectionModel import *
from fibsem.detection.utils import *

import matplotlib.pyplot as plt
import numpy as np

from PIL import Image
from random import shuffle

# print(os.path.dirname(model.__file__))

model_path = os.path.dirname(model.__file__)
model_filenames = glob.glob(
    os.path.join(model_path, "models", "boost*.pt")
)


# print(model_filenames)
detector = Detector(weights_file=model_filenames[0])
detector.detection_model.model.cuda()
# detector.detection_model.model.eval()

Unet(
  (encoder): ResNetEncoder(
    (conv1): Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track

In [16]:
detector.detection_model.model.parameters()

<generator object Module.parameters at 0x000001CE30C43BA0>

In [17]:
mem_params = sum([param.nelement()*param.element_size() for param in detector.detection_model.model.parameters()])
mem_bufs = sum([buf.nelement()*buf.element_size() for buf in detector.detection_model.model.buffers()])
mem = mem_params + mem_bufs # in bytes
print(mem * 1e-6, "MB")

57.335483999999994 MB
